#Intrazonal Trips
##Export Intrazonal Trips for each period alongside the Intazonal Distance for all TAZs


In [ ]:
from IPython.display import display
import os
import numpy as py
import pandas as pd
from functools import reduce
my_modeller = inro.modeller.Modeller()
emmebank = my_modeller.emmebank
desktop = my_modeller.desktop

#I/O 
main_directory = os.path.dirname(os.path.dirname(my_modeller.desktop.project.path)).replace("\\","/")
import_path = os.path.join(main_directory, "emme_project\Database").replace("\\","/")
ouput_path = os.path.join(main_directory, "output").replace("\\","/")
file_name_input = os.path.join(import_path, "xref_taz13_jur.csv")
#file_name_input2 = os.path.join(import_path, "SL_query.csv")

In [23]:
## Read Demand Matrices and aggreagted them in a single matrix(by TOD). Read the diagonal for intra trips
## Read distance Skim. Read diagnoal for intra skim.
## Summarize all in a single table for all TAZs

z=emmebank.scenario(100).zone_numbers
CR_data = pd.read_csv(file_name_input,sep=',')
#query = pd.read_csv(file_name_input2,sep=',')

#suffix = query['Query']
periods = ["EA", "AM", "MD", "PM", "EV"]
###import 
def Intrazonal_VMT ( period ) :
    matrix_id_list = []
    matrix_id_name = []
    ##choose selected demand matrices from emmebank and add them to get the total demand
    for matrix in emmebank.matrices():
        if matrix.type == "FULL" and period in matrix.description and "demand" in matrix.description and "Selected" not in matrix.description  :
            matrix_id_list.append(matrix.id)
        if matrix.type == "FULL" and "SOV MVOT GP distance" in matrix.description and period in matrix.description  :
            skim = emmebank.matrix( id = matrix.id).get_numpy_data()


    mat_intra = []
    for i in matrix_id_list :
        mat_intra.append(emmebank.matrix(id=i).get_numpy_data().diagonal())
    trip_intra = reduce(lambda x,y: x+y,  mat_intra)



    result = pd.DataFrame( { period: trip_intra , 'TAZ':z} )
    return result

for matrix in emmebank.matrices():

    if matrix.type == "FULL" and "SOV MVOT GP distance" in matrix.description and "AM" in matrix.description  :
        skim = emmebank.matrix( id = matrix.id).get_numpy_data()
        skim_intra = pd.DataFrame({ "Intra Distance": skim.diagonal() , 'TAZ':z}) 
        
temp = reduce(lambda x,y: pd.merge(x,y, on="TAZ" , how="left"), [skim_intra,Intrazonal_VMT("EA"),Intrazonal_VMT("AM"),Intrazonal_VMT("MD"),Intrazonal_VMT("PM"),Intrazonal_VMT("EV")] )
temp = temp[['TAZ', "Intra Distance", 'EA', 'AM', 'MD', 'PM', 'EV']]
temp["Intra Trips"] = temp[['EA', 'AM', 'MD', 'PM', 'EV']].sum(axis=1)
file_name_output = "Intrazonal_VMT.csv" 
dir_output = os.path.join(ouput_path, file_name_output)
temp.to_csv(dir_output, index= False)